In [67]:
import dash
from dash import dash_table, dcc, html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output

# Load your data
df = pd.read_csv('./results/results.csv')

# Define your input and output features
input_features = ["d_in", "d_out", "prec", "rf", "strategy"]
output_features = ["WorstLatency_hls", "IntervalMax_hls", "FF_hls", "LUT_hls", "BRAM_18K_hls", "DSP_hls","hls_synth_success"]

# Create a Dash application
app = dash.Dash(__name__)

# Create a table for the input features
input_table = dash_table.DataTable(
    data=df[input_features].to_dict('records'),
    columns=[{'name': i, 'id': i} for i in input_features],
    page_size=10,  # the number of rows visible on each page
)

# Create a table for the output features
output_table = dash_table.DataTable(
    data=df[output_features].to_dict('records'),
    columns=[{'name': i, 'id': i} for i in output_features],
    page_size=10,  # the number of rows visible on each page
)
marker_shapes = {'latency': 'x', 'resource': 'square'}

# Define the output features
output_features = ["IntervalMax_hls", "FF_hls", "LUT_hls", "BRAM_18K_hls", "DSP_hls", "hls_synth_success"]

# Initialize subplot object
fig = make_subplots(rows=2, cols=3, subplot_titles=output_features, vertical_spacing= 0.03, horizontal_spacing= 0.03, specs=[[{'type': 'scatter3d'}]*3]*2)

axis_string_keys = [str(n) for n in range(len(output_features))]
axis_string_keys[0] = ""

# Iterate over output features
for i, feature in enumerate(output_features):
    # Iterate over each strategy
    for strategy in ['latency', 'resource']:
        # Create a mask for the current strategy
        mask = df['strategy'] == strategy
        # Create 3D scatter plot
        scatter = go.Scatter3d(
            x=df['d_in'][mask],
            y=df['d_out'][mask],
            z=df[feature][mask],
            mode='markers',
            name=feature,  # Set the name of each trace to the feature name and strategy
            showlegend=False,
            marker=dict(
                size=5,
                color=df['rf'][mask],  # set color to an array/list of desired values
                colorscale='Viridis',  # choose a colorscale
                opacity=0.8,
                symbol=marker_shapes[strategy],  # Use different marker shapes for different strategies
                colorbar=dict(  # Add a colorbar to the marker
                    title="Reuse Factor",  # Title of the colorbar
                    titleside="right",  # Position of the title
                    thickness=15,  # Thickness of the colorbar
                    len=1  # Length of the colorbar
                )
            )
        )
        # Add scatter plot to subplot
        row = i // 3 + 1  # Integer division to get the row number
        col = i % 3 + 1  # Modulo operation to get the column number
        fig.add_trace(scatter, row=row, col=col)
        #fig['layout']['xaxis'+axis_string_keys[i]]['title']= "Input Size"
        #fig['layout']['yaxis'+axis_string_keys[i]]['title']= "Output Size"
        #fig['layout']['zaxis'+axis_string_keys[i]]['title']= feature
        scene=dict(
            xaxis_title="Input Size",
            yaxis_title="Output Size",
            zaxis_title=feature
        )
        fig.update_scenes(scene, row=row, col=col)
        #print("Added ", row, col)

# New code
for strategy in ['latency', 'resource']:
    scatter = go.Scatter(
        x=[None],  # No data for these traces
        y=[None],
        mode='markers',
        marker=dict(
            symbol=marker_shapes[strategy],
            size=10,
            opacity=0.7,
        ),
        name=strategy,  # The name of the strategy
        showlegend=True  # Show these traces in the legend
    )
    fig.add_trace(scatter)

fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    autosize=False,
    width=1800,
    height=1000,
    legend=dict(
        x=0,  # x position of the legend
        y=1,  # y position of the legend
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        )
    ),
)

fig.update_xaxes(showticklabels=False)  # Hide X-axis
fig.update_yaxes(showticklabels=False)  # Hide Y-axis

app.layout = html.Div([
        dcc.Graph(id='graph', figure=fig)  # Your Plotly figure

])


# Run the Dash application
if __name__ == '__main__':
    app.run_server(debug=True)